# LFSR

This was a pretty standard LFSR challenge, with the unusual twist that all the elements of `_p` are multiples of 8. Now, z3 tends to have a decent success rate with these, but for some reason it didn't work very well here. Nonetheless we attempted to run a few z3 instances over many hours before finally realising the key twist: You can define the taps over multiples of 8!

In particular, our feedback polynomial is $x^0+x^1+x^2+x^7+x^{128}$. But because of the [freshman's dream](https://en.wikipedia.org/wiki/Freshman%27s_dream), we can take this to the power of 8 to get $x^0+x^8+x^{16}+x^{56}+x^{1024}$, and now z3 can solve the problem very quickly (under 15 seconds).

In [1]:
from z3 import *
from Crypto.Cipher import AES
from math import prod
from base64 import b64decode

msg1 = '1q/f8g6idcDLqMnkXeA5N40HZSdOvN35tqGwttNzcLZkuNfxj6rNulIVWWmM+9fVPCBE/uhYSv5y6NUQN42ijHh2pyKpC/rPvzOjvt0ivEwEfHIHuydWfxhkP6KhzOKdQcPr9WWxxzTT9oxeb2sz/UThWYYXp9diaHlK+8aRvZ05Jyjz9s2Yvys048BW+0DEpvjZuGUla6yux68305JuNjx/BnKNW5pCz5uYcEE+q4e+7T5zn5HWcTH4fbEYeFhs0wTpXuzfL6Xa1ofJVPHK5z1tSBjHY+3Fiw7z2sad/f+om/prXJtc6oCFQIlv8WriNRzwhnnDeO97Vy4/63w/3zUD+jxzCsnDvfm1zQxD1Wu6plmJZM7yFffvCgjNspNZHY6juu+uvqHeZXnVQjJeD9x6sJ09riNucC/7U9T3mnxJbPl2M0e/Mwyisrd7/c7Pixy6rggXIQ94mu1GremMHD+XTsPZ/9Pr+5r0LEYttNb+srPJuPXbg64DD38Dp2nIL8Gy9q6o3//ZMpyTVVWb2spPSkSe9JAYDLYVUCDMMgHcls9Ibp6NS97J3LxXvKHetoN1JzWJ1mxSQCE4pXHga7pT+ewyW7DLLftW1fsRXz1w/rlbm/WTclW6H32uvi6kGuc1r6j3PuivEFCM1I/MZbiI4fxuZpEutY60KOznzy+UO74J2K1v0rlrb6K7TQeu1bZOODGNqpjGyrm/OvQXilz4TgRAsypC4DibX4Z3Hzccmt/RsVRVIyKWhywz+74Epr+VlShG8QATGMkdABkKKiWyq5sNgcMVAtq/7VP+bEfMfr1I04+Urs/3+R97R+MlZN+x5OJEHxuuN1CPsmW4AB6rdz7HFj0po0fydztLNDvZoZ+4X72sge/mOUBqv5dPneuSZ9xvFuVGkXKih6vbv23iUjie2rL6XN7aDcnY6xXsTnavVPe7+zzPLypn/l8mCO5LW2MftZvyc1MkfqKTHwb9ecSXF95r+0N5kXBPHdLV5WDsM+W4wHgeU1UW3qmGnvjGxVOUf2vojrFB/3lJw/qj9McgpWAvSZ3+o0BpoIMizqkZjK39DKRK2lVUy83M+NnOPF5Aq56yGuVJ7FAa/SheIf7bXwAXMuD8A+CqO3Qk3DRCfFlYT7iffIHsYl6kdD1372g3ZkQXEldF2pm2kmU+mW+1x/LlfzxQHdVdM57lwUHy67YSX5K7k2Ox8Ton4UKctXFrBvDusp/mFZR7MXJfOf9Mh96kUZ478uIR7XQlFXg+Q9bJWVaGfEcj6upR7vRTj6Ci67f11tRoBDkngpBV5QFWZIHc24ZGgQNY5nn/RX1Jcr4Jf+XC4RL52Jija/UBenHwZJOqgMXgeCSpww=='
msg2 = '86c1095277ce04268eb6c5d811669e5bf0616f68e23c504bcdc2aa7faa9c9b10'

In [2]:
enc = [int(i) for b in b64decode(msg1) for i in f'{b:08b}']

_t = [0, 1, 2, 7]
_p = [0, 8, 16, 32, 64, 120]
_f = [[0], [2], [3], [5], [0, 1], [0, 3], [1, 4], [2, 3],\
      [0, 1, 4], [2, 3, 4], [0, 1, 2, 4, 5], [1, 2, 3, 4, 5]]

bits = [BitVec(f'b{i}', 1) for i in range(len(enc))]

s = Solver()
for i in range(len(enc) - 1024):
    s.add(sum(bits[i+t*8] for t in _t) == bits[i+1024])
    s.add(sum(prod(bits[i+_p[j]] for j in x) for x in _f) == enc[i])
assert(s.check() == sat)

key_bits = ''.join(str(s.model()[b]) for b in bits[:128])
key = int(key_bits, 2).to_bytes(16, 'big')
cipher = AES.new(key, AES.MODE_CBC, iv=b"\0" * 16)
print(cipher.decrypt(bytes.fromhex(msg2)))

b'BALSN{almost_linear_too_easy}\x03\x03\x03'
